https://docs.sqlalchemy.org/en/20/tutorial/index.html#unified-tutorial

In [100]:
import sqlalchemy

In [101]:
sqlalchemy.__version__ 

'2.0.21'

https://docs.sqlalchemy.org/en/20/tutorial/engine.html

In [102]:
# the engine is the thing that connects to the DB : it is the echange point between code and DB (I think)

# here we connect to an in-memory-only SQLite database

from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)
#                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^ = URL
#                       ^^^^^                        = type of DB => important to choose the right Dialect (aka how to talk with each other ?)
#                              ^^^^^^^^              = DBAPI (Python Database API Specification) third party driver, if omitted sqlAlchemy will go to default
#                                          ^^^^^^^^  = DB location : here in memory (specific to sqlite)


In [103]:
engine

Engine(sqlite+pysqlite:///:memory:)

https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html

In [104]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())
    
# ROLLBACK at the end :
# The transaction is not committed automatically
# “autocommit” mode is available for special cases.


2023-10-31 21:37:10,540 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 21:37:10,544 INFO sqlalchemy.engine.Engine select 'hello world'
2023-10-31 21:37:10,545 INFO sqlalchemy.engine.Engine [generated in 0.00557s] ()
[('hello world',)]
2023-10-31 21:37:10,552 INFO sqlalchemy.engine.Engine ROLLBACK


In [105]:

# result object : 
# it’s best to ensure this object is consumed within the “connect” block,
# and is not passed along outside of the scope of our connection.
print(result.all())

[]


In [106]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE test_empty (x int, y int)"))

2023-10-31 21:37:28,920 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 21:37:28,921 INFO sqlalchemy.engine.Engine CREATE TABLE test_empty (x int, y int)
2023-10-31 21:37:28,922 INFO sqlalchemy.engine.Engine [generated in 0.00299s] ()
2023-10-31 21:37:28,924 INFO sqlalchemy.engine.Engine ROLLBACK


In [113]:
with engine.begin() as c:
    print("1")
    r = c.execute(text("SELECT x, y FROM test_empty2 WHERE x = :x"),{"x":1})
    print("2")
    print(r)
    print(r.first() is None)

2023-10-31 21:42:40,393 INFO sqlalchemy.engine.Engine BEGIN (implicit)
1
2023-10-31 21:42:40,396 INFO sqlalchemy.engine.Engine SELECT x, y FROM test_empty2 WHERE x = ?
2023-10-31 21:42:40,397 INFO sqlalchemy.engine.Engine [generated in 0.00134s] (1,)
2023-10-31 21:42:40,398 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: test_empty2
[SQL: SELECT x, y FROM test_empty2 WHERE x = ?]
[parameters: (1,)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [108]:
r

### commit as you go.

Use `engine.connect()` and then call `con.commit()` every time you commit a transaction.

You can define many transactions before commiting or commit after each and everyone

In [26]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()
    #  here we commit the changes(see below)

2023-10-31 12:46:46,733 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:46,734 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-10-31 12:46:46,734 INFO sqlalchemy.engine.Engine [generated in 0.00174s] ()
2023-10-31 12:46:46,736 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-10-31 12:46:46,737 INFO sqlalchemy.engine.Engine [generated in 0.00112s] [(1, 1), (2, 4)]
2023-10-31 12:46:46,740 INFO sqlalchemy.engine.Engine COMMIT


In [27]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM some_table"))
    print(result.all())
# will show [(1, 1), (2, 4)]

2023-10-31 12:46:46,948 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:46,949 INFO sqlalchemy.engine.Engine SELECT * FROM some_table
2023-10-31 12:46:46,951 INFO sqlalchemy.engine.Engine [generated in 0.00276s] ()
[(1, 1), (2, 4)]
2023-10-31 12:46:46,953 INFO sqlalchemy.engine.Engine ROLLBACK


In [28]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 2, "y": 2}, {"x": 3, "y": 3}],
    )
    #  here we DON'T commit the changes (see below)

2023-10-31 12:46:47,107 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:47,109 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-10-31 12:46:47,110 INFO sqlalchemy.engine.Engine [cached since 0.3749s ago] [(2, 2), (3, 3)]
2023-10-31 12:46:47,111 INFO sqlalchemy.engine.Engine ROLLBACK


In [29]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM some_table"))
    print(result.all())
# will show only [(1, 1), (2, 4)]

2023-10-31 12:46:47,332 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:47,333 INFO sqlalchemy.engine.Engine SELECT * FROM some_table
2023-10-31 12:46:47,334 INFO sqlalchemy.engine.Engine [cached since 0.3871s ago] ()
[(1, 1), (2, 4)]
2023-10-31 12:46:47,336 INFO sqlalchemy.engine.Engine ROLLBACK


###  begin once

Use `engine.begin()` only once.
* It will `COMMIT` at the end if the block is successful
* `ROLLBACK` if there is an error within the block

It is often preferred as it is more succinct and indicates the intention of the entire block up front.

In [30]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-10-31 12:46:48,040 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:48,041 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-10-31 12:46:48,043 INFO sqlalchemy.engine.Engine [cached since 1.307s ago] [(6, 8), (9, 10)]
2023-10-31 12:46:48,044 INFO sqlalchemy.engine.Engine COMMIT


In [31]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM some_table"))
    print(result.all())
# will show [(1, 1), (2, 4), (6, 8), (9, 10)]

2023-10-31 12:46:48,260 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:48,261 INFO sqlalchemy.engine.Engine SELECT * FROM some_table
2023-10-31 12:46:48,262 INFO sqlalchemy.engine.Engine [cached since 1.314s ago] ()
[(1, 1), (2, 4), (6, 8), (9, 10)]
2023-10-31 12:46:48,263 INFO sqlalchemy.engine.Engine ROLLBACK


In [32]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 10, "y": 18}, {"x": 19, "y": 100}],
    )
    raise BaseException("oups")

2023-10-31 12:46:48,483 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:48,485 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-10-31 12:46:48,487 INFO sqlalchemy.engine.Engine [cached since 1.752s ago] [(10, 18), (19, 100)]
2023-10-31 12:46:48,489 INFO sqlalchemy.engine.Engine ROLLBACK


BaseException: oups

In [33]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM some_table"))
    print(result.all())
# will show only [(1, 1), (2, 4), (6, 8), (9, 10)]

2023-10-31 12:46:48,673 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:48,674 INFO sqlalchemy.engine.Engine SELECT * FROM some_table
2023-10-31 12:46:48,675 INFO sqlalchemy.engine.Engine [cached since 1.728s ago] ()
[(1, 1), (2, 4), (6, 8), (9, 10)]
2023-10-31 12:46:48,678 INFO sqlalchemy.engine.Engine ROLLBACK


### result object

In [34]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x,y FROM some_table"))
    for row in result:
        print(f"\tx: {row.x} \t y: {row.y}")

2023-10-31 12:46:50,624 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:50,626 INFO sqlalchemy.engine.Engine SELECT x,y FROM some_table
2023-10-31 12:46:50,627 INFO sqlalchemy.engine.Engine [generated in 0.00225s] ()
	x: 1 	 y: 1
	x: 2 	 y: 4
	x: 6 	 y: 8
	x: 9 	 y: 10
2023-10-31 12:46:50,629 INFO sqlalchemy.engine.Engine ROLLBACK


In [35]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x,y FROM some_table"))
    for x,y in result:
        print(f"\tx: {x} \t y: {y}")

2023-10-31 12:46:50,828 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:50,830 INFO sqlalchemy.engine.Engine SELECT x,y FROM some_table
2023-10-31 12:46:50,830 INFO sqlalchemy.engine.Engine [cached since 0.2063s ago] ()
	x: 1 	 y: 1
	x: 2 	 y: 4
	x: 6 	 y: 8
	x: 9 	 y: 10
2023-10-31 12:46:50,832 INFO sqlalchemy.engine.Engine ROLLBACK


In [36]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT y,x FROM some_table"))
    for row in result:
        print(f"\tx?: {row.x} \t y?: {row.y}")
# here order does not matter but name does

2023-10-31 12:46:51,033 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:51,035 INFO sqlalchemy.engine.Engine SELECT y,x FROM some_table
2023-10-31 12:46:51,035 INFO sqlalchemy.engine.Engine [generated in 0.00192s] ()
	x?: 1 	 y?: 1
	x?: 2 	 y?: 4
	x?: 6 	 y?: 8
	x?: 9 	 y?: 10
2023-10-31 12:46:51,037 INFO sqlalchemy.engine.Engine ROLLBACK


In [37]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT y, x FROM some_table"))
    for x,y in result:
        print(f"\tx?: {x} \t y?: {y}")
# in this way the order matters

2023-10-31 12:46:51,299 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:51,301 INFO sqlalchemy.engine.Engine SELECT y, x FROM some_table
2023-10-31 12:46:51,302 INFO sqlalchemy.engine.Engine [generated in 0.00261s] ()
	x?: 1 	 y?: 1
	x?: 4 	 y?: 2
	x?: 8 	 y?: 6
	x?: 10 	 y?: 9
2023-10-31 12:46:51,303 INFO sqlalchemy.engine.Engine ROLLBACK


### sending parameters

In [38]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"\tx: {row.x}  y: {row.y}")

2023-10-31 12:46:53,108 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:46:53,111 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2023-10-31 12:46:53,111 INFO sqlalchemy.engine.Engine [generated in 0.00319s] (2,)
	x: 2  y: 4
	x: 6  y: 8
	x: 9  y: 10
2023-10-31 12:46:53,113 INFO sqlalchemy.engine.Engine ROLLBACK


## ORM session

In [ ]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"\tx: {row.x}  y: {row.y}")

In [ ]:
# also need to commit with sessions
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )
    session.commit()

## Metadata

The most common foundational objects for database metadata in SQLAlchemy are known as `MetaData`, `Table`, and `Column`. 

In [45]:
# hand-create a metadata object

from sqlalchemy import MetaData
metadata_obj = MetaData()

In [46]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [47]:
# user_table.c : about Columns of the Table user_table
user_table.c.keys()

['id', 'name', 'fullname']

In [48]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [49]:
user_table.c.fullname

Column('fullname', String(), table=<user_account>)

**DDL**\
An acronym for Data Definition Language. DDL is the subset of SQL that relational databases use to configure tables, constraints, and other permanent objects within a database schema. SQLAlchemy provides a rich API for constructing and emitting DDL expressions.

### Constraints
are listed after name and type
1. primary key: `primary_key=True` 
2. foreign key: `ForeignKey("table.column)` can replace the type which is infered from the table.column
3. not null: `nullable=False`

In [50]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

#### Foreign key constraints

In [51]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

### Emitting DDl to database

In [52]:
# need an engine for it
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [98]:
metadata_obj.create_all(engine)

2023-10-31 14:22:09,989 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 14:22:09,991 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-10-31 14:22:09,991 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 14:22:09,993 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-10-31 14:22:09,994 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 14:22:09,995 INFO sqlalchemy.engine.Engine COMMIT


In [99]:
metadata_obj.drop_all(engine)

2023-10-31 14:22:32,749 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 14:22:32,750 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-10-31 14:22:32,751 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 14:22:32,753 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-10-31 14:22:32,754 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 14:22:32,756 INFO sqlalchemy.engine.Engine 
DROP TABLE address
2023-10-31 14:22:32,757 INFO sqlalchemy.engine.Engine [no key 0.00103s] ()
2023-10-31 14:22:32,758 INFO sqlalchemy.engine.Engine 
DROP TABLE user_account
2023-10-31 14:22:32,759 INFO sqlalchemy.engine.Engine [no key 0.00076s] ()
2023-10-31 14:22:32,760 INFO sqlalchemy.engine.Engine COMMIT


Migration tools are usually appropriate

Overall, the CREATE / DROP feature of MetaData is useful for test suites, small and/or new applications, and applications that use short-lived databases. For management of an application database schema over the long term however, a schema management tool such as Alembic, which builds upon SQLAlchemy, is likely a better choice, as it can manage and orchestrate the process of incrementally altering a fixed database schema over time as the design of the application changes.

https://alembic.sqlalchemy.org/en/latest/

### ORM declarative table (ORM mapped class)

PEP 484
https://peps.python.org/pep-0484/


In [1]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

In [2]:
Base.metadata

MetaData()

In [6]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import String
from sqlalchemy import ForeignKey

In [4]:
class User(Base):
    __tablename__ = "user_account"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]   # can be null
        
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}), name={self.name!r}, fullname={self.fullname!r})"

In [7]:
class Address(Base):
    __tablename__ = "address"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    email_address: Mapped[str]
        
    user: Mapped[User] = relationship(back_populates="addresses")
        
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [8]:
sandy = User(name="sandy", fullname="Sandy Cheeks")

In [9]:
sandy

User(id=None), name='sandy'

In [10]:
Base.metadata

MetaData()

In [11]:
Base.registry

In [14]:
Base.metadata.sorted_tables

[Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>, nullable=False), Column('fullname', String(), table=<user_account>), schema=None),
 Table('address', MetaData(), Column('id', Integer(), table=<address>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('user_account.id'), table=<address>, nullable=False), Column('email_address', String(), table=<address>, nullable=False), schema=None)]

In [15]:
# need an engine for it
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [16]:
Base.metadata.create_all(engine)

2023-10-31 12:42:27,227 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:42:27,229 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-10-31 12:42:27,230 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 12:42:27,232 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-10-31 12:42:27,235 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 12:42:27,236 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-10-31 12:42:27,237 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 12:42:27,238 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2023-10-31 12:42:27,251 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 12:42:27,253 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2023-10-31 12:42:27,255 INFO sqlalchemy.engine.Engine [no key 0.00153s] ()
2023-10-31 12:42:27,256 INFO sqlalchemy.engine.

#### table reflection : from data table to schema (?)

In [20]:
from sqlalchemy import Table
from sqlalchemy import MetaData


In [39]:
metadata_obj = MetaData()


In [40]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2023-10-31 12:47:04,691 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 12:47:04,693 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2023-10-31 12:47:04,695 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 12:47:04,695 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2023-10-31 12:47:04,696 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2023-10-31 12:47:04,697 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2023-10-31 12:47:04,698 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 12:47:04,699 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2023-10-31 12:47:04,699 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 12:47:04,700 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type i

In [43]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

# Manipulating data

In [54]:
from sqlalchemy import insert
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")

In [55]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [56]:
compiled = stmt.compile()

In [59]:
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [63]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2023-10-31 13:35:53,776 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 13:35:53,778 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-10-31 13:35:53,779 INFO sqlalchemy.engine.Engine [generated in 0.00243s] ('spongebob', 'Spongebob Squarepants')
2023-10-31 13:35:53,779 INFO sqlalchemy.engine.Engine COMMIT


In [64]:
result.inserted_primary_key

(1,)

In [67]:
stmt2 = insert(user_table).values(name="clark", fullname="Clark Superman")
with engine.connect() as conn:
    result = conn.execute(stmt2)
    conn.commit()
    print(result.inserted_primary_key)

2023-10-31 13:37:04,239 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 13:37:04,240 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-10-31 13:37:04,242 INFO sqlalchemy.engine.Engine [cached since 70.47s ago] ('clark', 'Clark Superman')
2023-10-31 13:37:04,243 INFO sqlalchemy.engine.Engine COMMIT
(3,)


In [70]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()
    print(result.inserted_primary_key_rows)

2023-10-31 13:39:45,752 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 13:39:45,753 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-10-31 13:39:45,755 INFO sqlalchemy.engine.Engine [cached since 27.78s ago] [('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star')]
2023-10-31 13:39:45,755 INFO sqlalchemy.engine.Engine COMMIT
[(None,), (None,)]


In [71]:
# advanced alchemy T.T

from sqlalchemy import select, bindparam
scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {
                "username": "spongebob",
                "email_address": "spongebob@sqlalchemy.org",
            },
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2023-10-31 13:44:49,042 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 13:44:49,043 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2023-10-31 13:44:49,045 INFO sqlalchemy.engine.Engine [generated in 0.00259s] [('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org')]
2023-10-31 13:44:49,046 INFO sqlalchemy.engine.Engine COMMIT


In [72]:
insert_stmt = insert(address_table).returning(
    address_table.c.id, address_table.c.email_address
)
print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address



#### insert from select 
when want to copy data from one part of the database into a new set of rows without fetching and re-sending the data from the client

In [73]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


In [74]:
with engine.connect() as conn:
    result = conn.execute(insert_stmt.returning(address_table.c.id, address_table.c.email_address))
    conn.commit()
    print(result)

2023-10-31 13:50:59,567 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 13:50:59,568 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || ? AS anon_1 
FROM user_account RETURNING id, email_address
2023-10-31 13:50:59,569 INFO sqlalchemy.engine.Engine [generated in 0.00317s] ('@aol.com',)
2023-10-31 13:50:59,571 INFO sqlalchemy.engine.Engine COMMIT


OperationalError: (sqlite3.OperationalError) cannot commit transaction - SQL statements in progress
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [78]:
result.keys()

RMKeyView(['id', 'email_address'])

In [83]:
result.all()

[(4, 'spongebob@aol.com'),
 (5, 'spongebob@aol.com'),
 (6, 'clark@aol.com'),
 (7, 'sandy@aol.com'),
 (8, 'patrick@aol.com'),
 (9, 'sandy@aol.com'),
 (10, 'patrick@aol.com')]

In [84]:
from sqlalchemy import select

In [85]:
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [86]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2023-10-31 13:58:48,933 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 13:58:48,934 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2023-10-31 13:58:48,935 INFO sqlalchemy.engine.Engine [generated in 0.00199s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
(2, 'spongebob', 'Spongebob Squarepants')
2023-10-31 13:58:48,936 INFO sqlalchemy.engine.Engine ROLLBACK


In [88]:
from sqlalchemy.orm import Session

In [89]:
stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2023-10-31 14:00:06,783 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 14:00:06,788 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2023-10-31 14:00:06,790 INFO sqlalchemy.engine.Engine [generated in 0.00239s] ('spongebob',)
(User(id=1), name='spongebob',)
(User(id=2), name='spongebob',)
2023-10-31 14:00:06,794 INFO sqlalchemy.engine.Engine ROLLBACK


In [90]:
# the FROM is infered
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [91]:
# the FROM is infered
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [92]:
# use a tuple of string names
print(select(user_table.c["name", "fullname"]))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [94]:
my_c_tupple = ("name", "fullname")
print(select(user_table.c[my_c_tupple]))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [95]:
# with ORM entities
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [96]:
# with ORM : each row is a single element (of the class, I think) (one element per class)
row = session.execute(select(User)).first()
row

2023-10-31 14:05:23,911 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 14:05:23,913 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2023-10-31 14:05:23,914 INFO sqlalchemy.engine.Engine [generated in 0.00120s] ()


(User(id=1), name='spongebob',)

In [97]:
user = session.scalars(select(User)).first()
user

2023-10-31 14:07:26,048 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2023-10-31 14:07:26,049 INFO sqlalchemy.engine.Engine [cached since 122.1s ago] ()


User(id=1), name='spongebob'